In [1]:
# 1. Install necessary libraries (if not already installed)
!pip install transformers datasets accelerate evaluate

import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00


In [2]:
# 2. Load the Dataset
# We use the dataset you selected: wikitext
dataset = load_dataset("vesteinn/wikitext-220728-250728")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/15.7M [00:00<?, ?B/s]

validation.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/9007 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1399 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1418 [00:00<?, ? examples/s]

In [3]:
# 3. Load Tokenizer & Model (GPT-2)
# We use GPT-2 because WikiText is a text generation dataset.
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
# GPT-2 does not have a padding token by default, so we set it to the EOS token
tokenizer.pad_token = tokenizer.eos_token

In [5]:
# 4. Tokenization Function
# This prepares the text for the model.
# We typically set max_length to 128 or 256 for quick fine-tuning demos.
def tokenize_function(examples):
    outputs = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    # For text generation, the "labels" (what we predict) are the same as the input
    outputs["labels"] = outputs["input_ids"].copy()
    return outputs

# Apply tokenization to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9007 [00:00<?, ? examples/s]

Map:   0%|          | 0/1399 [00:00<?, ? examples/s]

Map:   0%|          | 0/1418 [00:00<?, ? examples/s]

In [6]:
# 5. Data Collator
# This handles batching and padding during training
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [7]:
# 6. Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Reduced to 8 to avoid memory issues on Colab
    per_device_eval_batch_size=8,
    num_train_epochs=1,             # Set to 1 for a quick test run (increase to 3 later)
    weight_decay=0.01,
    eval_strategy="steps",          # Evaluate every few steps
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    push_to_hub=False,
    report_to="none"                # Disables wandb/tensorboard for simplicity
)

In [8]:
# 7. Initialize the Trainer (Removed 'tokenizer' argument to fix TypeError)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
)

In [9]:
# 8. Start Training
print("Starting training...")
trainer.train()

Starting training...


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
500,3.501245,3.437374
1000,3.525054,3.416318


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=1126, training_loss=3.5571710373114525, metrics={'train_runtime': 411.848, 'train_samples_per_second': 21.87, 'train_steps_per_second': 2.734, 'total_flos': 588364333056000.0, 'train_loss': 3.5571710373114525, 'epoch': 1.0})

In [10]:
import math

# Run the evaluation
eval_results = trainer.evaluate()

# Calculate Perplexity
perplexity = math.exp(eval_results['eval_loss'])

print(f"Evaluation Loss: {eval_results['eval_loss']:.4f}")
print(f"Perplexity: {perplexity:.4f}")

Evaluation Loss: 3.4152
Perplexity: 30.4216
